In [77]:
import getpass
import requests
import sys
import http.client as http_client
import logging
import json
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances_argmin, pairwise_distances
pd.set_option('mode.chained_assignment', None)

In [4]:
user = "Hackathon"

pw = getpass.getpass("Login password for user '{}': ".format(user))

url = "https://ssl.biomax.de/nanocommons/bioxm/rest/api"

proxies = {
  #'https': 'server:8080'
}

print("Opening session...")
response = requests.get(url + "/createUserSessionSimple?name={}&password={}".format(user, pw),
                        proxies=proxies)

session_id = ""
if response.status_code == 200:
    session_id = response.text
    print("Opened session " + session_id)
else:
    print("Failed to open session: " + str(response.text))
    sys.exit(1)

request_data= """
  {
    "SearchResultWithReportsRequest":
    {
      "firstIndex": 0,
      "maxCount": 100,
      "local": False,
      "viewName": "ivan",
      "query": {
          "queryName": "ivan"
      }
    }
  }
"""

try:               
    headers = {'Content-type': 'application/json'}

    http_client.HTTPConnection.debuglevel = 1

    logging.basicConfig()
    logging.getLogger().setLevel(logging.DEBUG)
    requests_log = logging.getLogger("requests.packages.urllib3")
    requests_log.setLevel(logging.DEBUG)
    requests_log.propagate = True

    response = requests.post(url + "/getSearchResultWithReports?userSessionId=" + session_id, data=request_data,
                           headers=headers, proxies=proxies)
    print("Status: " + str(response.status_code))

    if (response.status_code == 200):
        print(response.text)
        json_data2 = json.loads(response.text)
        json_dataset2 = json_data2['SearchResultWithReports']['objectReports']

    else:
        print("ERROR:")
        print(response.text)
 

finally:
    response = requests.get(url + "/destroyUserSession?userSessionId=" + session_id, proxies=proxies)
    if (response.status_code == 200):
        print("\nClosed session.")
    else:
        print("\nError: Failed to close session " + session_id)
        
        
data_dict = {}
data_dict['Id'] = []
for dato in json_data2['SearchResultWithReports']['objectReports']:
    for atr in dato['attributes']:
        data_dict[atr['name']] = []

        
  
for dato in json_data2['SearchResultWithReports']['objectReports']:
    data_dict['Id'].append(dato["objectURL"])
    for key, value in data_dict.items():
        for atr in dato['attributes']: 
            if atr['name'] == key:
                try:
                    nr = atr['nestedReports']
                    data_dict[key].append(nr['attributes']['values'])
                except KeyError:
                    data_dict[key].append(None)
                    continue

    
df = pd.DataFrame(data_dict)
print(df)

Login password for user 'Hackathon': ········
Opening session...


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): ssl.biomax.de:443


Opened session 036b576ccf7930c5201569bd17560d157bb8e9a8
send: b'POST /nanocommons/bioxm/rest/api/getSearchResultWithReports?userSessionId=036b576ccf7930c5201569bd17560d157bb8e9a8 HTTP/1.1\r\nHost: ssl.biomax.de\r\nUser-Agent: python-requests/2.21.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\nContent-type: application/json\r\nContent-Length: 208\r\n\r\n'
send: b'\n  {\n    "SearchResultWithReportsRequest":\n    {\n      "firstIndex": 0,\n      "maxCount": 100,\n      "local": False,\n      "viewName": "ivan",\n      "query": {\n          "queryName": "ivan"\n      }\n    }\n  }\n'


DEBUG:urllib3.connectionpool:https://ssl.biomax.de:443 "POST /nanocommons/bioxm/rest/api/getSearchResultWithReports?userSessionId=036b576ccf7930c5201569bd17560d157bb8e9a8 HTTP/1.1" 200 None


reply: 'HTTP/1.1 200 OK\r\n'
header: Date header: Server header: Strict-Transport-Security header: X-Frame-Options header: X-Content-Type-Options header: Content-Type header: Keep-Alive header: Connection header: Transfer-Encoding Status: 200


DEBUG:chardet.charsetprober:utf-8  confidence = 0.99
DEBUG:chardet.charsetprober:SHIFT_JIS Japanese confidence = 0.01
DEBUG:chardet.charsetprober:EUC-JP Japanese confidence = 0.01
DEBUG:chardet.charsetprober:GB2312 Chinese confidence = 0.01
DEBUG:chardet.charsetprober:EUC-KR Korean confidence = 0.01
DEBUG:chardet.charsetprober:CP949 Korean confidence = 0.01
DEBUG:chardet.charsetprober:Big5 Chinese confidence = 0.01
DEBUG:chardet.charsetprober:EUC-TW Taiwan confidence = 0.01
DEBUG:chardet.charsetprober:windows-1251 Russian confidence = 0.01
DEBUG:chardet.charsetprober:KOI8-R Russian confidence = 0.01
DEBUG:chardet.charsetprober:ISO-8859-5 Russian confidence = 0.0
DEBUG:chardet.charsetprober:MacCyrillic Russian confidence = 0.01
DEBUG:chardet.charsetprober:IBM866 Russian confidence = 0.01
DEBUG:chardet.charsetprober:IBM855 Russian confidence = 0.01
DEBUG:chardet.charsetprober:ISO-8859-7 Greek confidence = 0.01
DEBUG:chardet.charsetprober:windows-1253 Greek confidence = 0.01
DEBUG:chardet

{"SearchResultWithReports":{"totalCount":16,"objectReports":[{"objectURL":"Experiment:Hackathon:HA_NP00262","attributes":[{"name":"Particle","type":"object","error":false,"forbidden":false,"values":"NP00262"},{"name":"..Type","type":"report","error":false,"forbidden":false,"nestedReports":{"objectURL":"ExperimentDataEntry:Hackathon:HA_NP00262:{Element:Variable:VeN0022}","attributes":{"name":"_","type":"text","error":false,"forbidden":false,"values":"Si"}}},{"name":"..Coating","type":"report","error":false,"forbidden":false,"nestedReports":{"objectURL":"ExperimentDataEntry:Hackathon:HA_NP00262:{Element:Variable:VeN0023}","attributes":{"name":"_","type":"text","error":false,"forbidden":false,"values":"aminoacid"}}},{"name":"..Type of Coating","type":"report","error":false,"forbidden":false,"nestedReports":{"objectURL":"ExperimentDataEntry:Hackathon:HA_NP00262:{Element:Variable:VeN0024}","attributes":{"name":"_","type":"text","error":false,"forbidden":false,"values":"neutral"}}},{"name":"

DEBUG:chardet.charsetprober:utf-8  confidence = 0.99
DEBUG:chardet.charsetprober:SHIFT_JIS Japanese confidence = 0.01
DEBUG:chardet.charsetprober:EUC-JP Japanese confidence = 0.01
DEBUG:chardet.charsetprober:GB2312 Chinese confidence = 0.01
DEBUG:chardet.charsetprober:EUC-KR Korean confidence = 0.01
DEBUG:chardet.charsetprober:CP949 Korean confidence = 0.01
DEBUG:chardet.charsetprober:Big5 Chinese confidence = 0.01
DEBUG:chardet.charsetprober:EUC-TW Taiwan confidence = 0.01
DEBUG:chardet.charsetprober:windows-1251 Russian confidence = 0.01
DEBUG:chardet.charsetprober:KOI8-R Russian confidence = 0.01
DEBUG:chardet.charsetprober:ISO-8859-5 Russian confidence = 0.0
DEBUG:chardet.charsetprober:MacCyrillic Russian confidence = 0.01
DEBUG:chardet.charsetprober:IBM866 Russian confidence = 0.01
DEBUG:chardet.charsetprober:IBM855 Russian confidence = 0.01
DEBUG:chardet.charsetprober:ISO-8859-7 Greek confidence = 0.01
DEBUG:chardet.charsetprober:windows-1253 Greek confidence = 0.01
DEBUG:chardet

send: b'GET /nanocommons/bioxm/rest/api/destroyUserSession?userSessionId=036b576ccf7930c5201569bd17560d157bb8e9a8 HTTP/1.1\r\nHost: ssl.biomax.de\r\nUser-Agent: python-requests/2.21.0\r\nAccept-Encoding: gzip, deflate\r\nAccept: */*\r\nConnection: keep-alive\r\n\r\n'
reply: 'HTTP/1.1 200 OK\r\n'
header: Date header: Server header: Strict-Transport-Security header: X-Frame-Options header: X-Content-Type-Options header: Content-Type header: Keep-Alive header: Connection header: Transfer-Encoding 
Closed session.
                                  Id Particle ..Type               ..Coating  \
0    Experiment:Hackathon:HA_NP00262     None     Si               aminoacid   
1    Experiment:Hackathon:HA_NP00266     None     Si              aminoalkyl   
2    Experiment:Hackathon:HA_NP00269     None     Si               aminoacid   
3    Experiment:Hackathon:HA_NP00255     None     Ti                uncoated   
4    Experiment:Hackathon:HA_NP00256     None     Ti                     PVP   
5   

In [1]:
from sklearn.metrics import pairwise_distances

In [34]:
df_numeric = df.select_dtypes(exclude=['object'])
# print(list(df))
# print(df[ '..Shape'])

print(np.unique(df[ '..Shape']))

['Cubic' 'Nanorods' 'Spherical' 'Spherical/ Square' 'Square'
 'Square/Spherical/Rods' 'Various Shapes' 'Various shapes']


In [22]:
# print(df_numeric)
df_num_nona = df_numeric.dropna()
print(df_num_nona)

Empty DataFrame
Columns: [_.Nominal Size (nm), _.DLS (nm), _.DLS SD (nm), _.PDI, _.PDI SD, _.Zeta (mV), _.Zeta SD (mV), _.Electrophoretic mobiliy (µmcm/Vs), _.Electrophoretic mobiliy SD (µmcm/Vs), _.TEM (nm), _.TEM SD (nm), _.TEM Shortest (nm), _.TEM Shortest SD (nm), _.TEM Longest (nm), _.TEM Longest SD (nm), _.STEM (nm), _.STEM SD (nm), _.BET (m^2/g), _.BET SD(m^2/g), _.Energy Band Gap (eV), _.Geometric Surface Area (nm2)]
Index: []

[0 rows x 21 columns]


In [18]:
pairwise_distances(df_num_nona, df_num_nona, metric='manhattan')

NameError: name 'df_num_nona' is not defined

In [27]:
print(df_numeric.dropna(how='all'))

    _.Nominal Size (nm)  _.DLS (nm)  _.DLS SD (nm)  _.PDI  _.PDI SD  \
0                    20       28.90       25.19000  0.110     0.005   
1                    20       23.48        0.08185  0.172     0.015   
2                    20       25.24        0.77130  0.174     0.041   
3                    10     1325.00       74.47000  0.277     0.079   
4                    10     3185.00      181.60000  0.240     0.097   
5                    10     4391.00      287.40000  0.172     0.115   
6                    10     3109.00      140.50000  0.177     0.106   
7                    20     1609.00      917.80000  0.895     0.128   
8                    20      403.00       51.32000  0.810     0.107   
9                    70      172.40        1.21200  0.183     0.009   
10                   20      269.50        8.97900  0.291     0.024   
11                   33      257.70       14.32000  0.510     0.082   
12                  150      411.80        7.11200  0.239     0.021   
13    

                                  Id Particle ..Type               ..Coating  \
0    Experiment:Hackathon:HA_NP00262     None     Si               aminoacid   
1    Experiment:Hackathon:HA_NP00266     None     Si              aminoalkyl   
2    Experiment:Hackathon:HA_NP00269     None     Si               aminoacid   
3    Experiment:Hackathon:HA_NP00255     None     Ti                uncoated   
4    Experiment:Hackathon:HA_NP00256     None     Ti                     PVP   
5    Experiment:Hackathon:HA_NP00257     None     Ti                    F127   
6    Experiment:Hackathon:HA_NP00258     None     Ti                  AA4040   
7    Experiment:Hackathon:HA_NP00259     None     Ti                uncoated   
8    Experiment:Hackathon:HA_NP00260     None     Ti                uncoated   
9    Experiment:Hackathon:HA_NP00254     None     Ti                uncoated   
10   Experiment:Hackathon:HA_NP00193     None     Ce                uncoated   
11   Experiment:Hackathon:HA_NP00192    

In [474]:
def all_numerical(df):
    dfs = df.select_dtypes(include=['object'])
    df_numeric = df.select_dtypes(exclude=['object'])
    dfsc = pd.DataFrame()
    string_maps = []
    for val in list(dfs):
        if dfs[val].isnull().values.any() == False:
            i = 0
            uniques = np.unique(dfs[val])
            u, indices = np.unique(dfs[val], return_index=True)
            if val == 'Id':
                ids = {}
                for i in range(len(indices)):
                    ids[u[i]] = indices[i]
            data = np.searchsorted(uniques, dfs[val])
#             print(data)
            dft = pd.Series(data).rename(val)
#             print(dft)
            df_numeric[val] = dft.values
            dicts = {}
            for i in range(len(u)):
                dicts[u[i]] = dft.loc[indices[i]]
            string_maps.append({val:dicts})
#             for i in u:
#                 print(i)
#                 dict[u[i]]=dft.loc[[i]]
#             print(dict)
    qj = {}
    for key in list(df_numeric):
        qj[key] = None
    return df_numeric, ids ,qj, string_maps

In [475]:
all_numerical(df)

(    _.Nominal Size (nm)  _.DLS (nm)  _.DLS SD (nm)  _.PDI  _.PDI SD  \
 0                    20       28.90       25.19000  0.110     0.005   
 1                    20       23.48        0.08185  0.172     0.015   
 2                    20       25.24        0.77130  0.174     0.041   
 3                    10     1325.00       74.47000  0.277     0.079   
 4                    10     3185.00      181.60000  0.240     0.097   
 5                    10     4391.00      287.40000  0.172     0.115   
 6                    10     3109.00      140.50000  0.177     0.106   
 7                    20     1609.00      917.80000  0.895     0.128   
 8                    20      403.00       51.32000  0.810     0.107   
 9                    70      172.40        1.21200  0.183     0.009   
 10                   20      269.50        8.97900  0.291     0.024   
 11                   33      257.70       14.32000  0.510     0.082   
 12                  150      411.80        7.11200  0.239     0

In [478]:
def prepare_query(df):
    df, ids, qj, dicts = all_numerical(df)
    print("String have been converted to unique integers as follows: ")
    for di in dicts:
        print(di)
    print("Import the query values needed on the following object: ")
    print(qj)

In [495]:
prepare_query(df)

String have been converted to unique integers as follows: 
{'Id': {'Experiment:Hackathon:HA_NP00192': 0, 'Experiment:Hackathon:HA_NP00193': 1, 'Experiment:Hackathon:HA_NP00214a': 2, 'Experiment:Hackathon:HA_NP00214b': 3, 'Experiment:Hackathon:HA_NP00254': 4, 'Experiment:Hackathon:HA_NP00255': 5, 'Experiment:Hackathon:HA_NP00256': 6, 'Experiment:Hackathon:HA_NP00257': 7, 'Experiment:Hackathon:HA_NP00258': 8, 'Experiment:Hackathon:HA_NP00259': 9, 'Experiment:Hackathon:HA_NP00260': 10, 'Experiment:Hackathon:HA_NP00262': 11, 'Experiment:Hackathon:HA_NP00266': 12, 'Experiment:Hackathon:HA_NP00269': 13, 'Experiment:Hackathon:HA_NP00282': 14, 'Experiment:Hackathon:HA_NP00283': 15}}
{'..Type': {'Ag': 0, 'Ce': 1, 'Si': 2, 'Ti': 3, 'Zn': 4}}
{'..Coating': {'AA4040': 0, 'F127': 1, 'PVP': 2, 'aminoacid': 3, 'aminoalkyl': 4, 'triethoxycaprilysilane': 5, 'uncoated': 6}}
{'..Type of Coating': {'negative': 0, 'neutral': 1, 'positive': 2, 'uncoated': 3}}
{'..Shape': {'Cubic': 0, 'Nanorods': 1, 'Spheric

In [486]:
def similarity_search(df, query_dict, metric, bring):
    search_for = []
    for key in query_dict:
        search_for.append(key)
    dfn, ids, qj, dicts = all_numerical(df)
    sfdf = dfn[search_for].dropna()
    query_dict_for_df = {}
    for key in query_dict:
        query_dict_for_df[key] = [query_dict[key]]
    search_for_dataset = pd.DataFrame.from_dict(query_dict_for_df)
    if metric is not None:
        pwd = pairwise_distances(search_for_dataset, sfdf, metric=metric)
    else:
        pwd = pairwise_distances(search_for_dataset, sfdf)
    indexes = sfdf.index.values.astype(int)
    df_shorted = pd.DataFrame()
    pwd_length = len(pwd[0])
    pwdm = pwd[0]
    mins_shorted = []
    for i in range(pwd_length):
        minElement = np.amin(pwdm)
        result = np.where(pwdm == np.amin(pwdm))
        pwdm = np.delete(pwdm, result[0][0], 0)
        mins_shorted.append(minElement)
    ind_uniq = []
    for s in mins_shorted:
        ind = np.where(pwd[0] == s)
        for indi in ind[0]:
            if indi in ind_uniq:
                continue
            else:
                ind_uniq.append(indi)
    df_similar = pd.DataFrame(columns=list(df))
    ids = []
    for indic in ind_uniq:
        df_similar.append(df.loc[[indic]])
        ids.append(df.loc[[indic]]['Id'])
    return ids[:bring], mins_shorted[:bring]

#     print(mins_shorted)
#     print(df_similar)
#     print(df.loc[[indexes[result[0][0]]]])
#     print("indice is: " + str(indexes[result[0][0]]))
#     print(minElement)
#     print("min element at: " + str(result[1][0]))
#     print("indice is: " + str(indexes[result[1][0]]))
#     print(df.loc[[indexes[result[1][0]]]]['Id'])
#     print(pwd)

In [487]:
query = {'_.Nominal Size (nm)':2, '_.TEM (nm)':19.6}

In [494]:
similarity_search(df, query, "cosine", 20)

([6    Experiment:Hackathon:HA_NP00258
  Name: Id, dtype: object, 7    Experiment:Hackathon:HA_NP00259
  Name: Id, dtype: object, 3    Experiment:Hackathon:HA_NP00255
  Name: Id, dtype: object, 0    Experiment:Hackathon:HA_NP00262
  Name: Id, dtype: object, 1    Experiment:Hackathon:HA_NP00266
  Name: Id, dtype: object, 2    Experiment:Hackathon:HA_NP00269
  Name: Id, dtype: object, 5    Experiment:Hackathon:HA_NP00257
  Name: Id, dtype: object, 4    Experiment:Hackathon:HA_NP00256
  Name: Id, dtype: object],
 [0.1303935162605535,
  0.1303935162605535,
  0.13315390586467668,
  0.2311847696072329,
  0.2311847696072329,
  0.2311847696072329,
  0.4101338839016322,
  0.5643313622874633])